In [2]:
import soundfile as sf
import numpy as np
import scipy as signal
import matplotlib.pyplot as plt
import math
import pandas as pd
import itertools
from functools import lru_cache

In [2]:
# Load data
data1,samplerate = sf.read('C:/Users/lenovo/Desktop/ODAS_4223_0108/postfiltered_2020-01-08_09_25_03_1.raw', 
               channels=4, 
               samplerate=16000,
               subtype='PCM_16'
              )
data2,samplerate = sf.read('C:/Users/lenovo/Desktop/ODAS_4223_0108/postfiltered_2020-01-08_09_25_03_2.raw', 
               channels=4, 
               samplerate=16000,
               subtype='PCM_16'
              )

In [3]:
#Functions
def truncate(sig1, sig2):
    '''
    This function truncates the longer signal and return two signals of the same length
    '''
    l1, l2 = len(sig1), len(sig2)
    if l1 <= l2:
        return sig1, sig2[0:l1]
    else:
        return sig1[0:l2], sig2

def prepadding(sig1, sig2, winDuration, Fs):
    l = winDuration * Fs
    N = len(sig1)    
    # Padding to have an interger number of windows
    if N%l != 0:
        padWidth = l-N%l
        sig1 = np.pad(sig1, (0, padWidth), 'constant', constant_values = 0)
        sig2 = np.pad(sig2, (0, padWidth), 'constant', constant_values = 0)
        
    numOfWindows = len(sig1)/l
    
    return sig1, sig2, numOfWindows

def whiten(sig):
    return sig/np.abs(sig)

def gcc_phat(sig1, sig2, fs=16000, max_tau=None, interp=1, window=True, windowName="hamming"):
    '''
    This function computes the offset between the signal sig and the reference signal refsig
    using the Generalized Cross Correlation - Phase Transform (GCC-PHAT)method. In this modified
    function, sig1 and sig2 are garuanteed to have the same length
    '''
    # Make sure the length for the FFT is larger or equal than len(sig) + len(refsig)
    # n1, n2 = sig1.shape[0], sig2.shape[0]
    n = sig1.shape[0]
    
    # Add window
    if window: 
        win1 = getattr(np, windowName)(n)
        win2 = getattr(np, windowName)(n)
        sig1 = sig1 * win1
        sig2 = sig2 * win2

    # Generalized Cross Correlation Phase Transform
    SIG1 = np.fft.rfft(sig1, n=n)
    SIG2 = np.fft.rfft(sig2, n=n)
    
    W1 = whiten(SIG1)
    W2 = whiten(SIG2)
    
    R = W1 * np.conj(W2)
    
    cc = np.fft.irfft(R, n=(interp * n))

    # cc = np.fft.irfft(R / np.abs(R), n=(interp * n))

    max_shift = int(interp * n / 2)
    if max_tau:
        max_shift = np.minimum(int(interp * fs * max_tau), max_shift)

    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))

    # find max cross correlation index
    shift = np.argmax(np.abs(cc)) - max_shift

    tau = np.float(shift / float(interp * fs))
    
    if(np.any(np.isnan(cc))): 
        tau = math.nan
    
    return tau, cc, R, SIG1, SIG2


def width_cc(cc, threshold=0.5):
    '''
    Calculate the width between the first sample and the last sample that are above the thresold.
    Threshold is precentage of the peak value
    '''
    cc_max = np.amax(cc)
    peak_index = np.where(cc == cc_max)
    mag_thr = threshold * cc_max
    width = 0
    for ii in range(len(cc)):
        if cc[ii] > mag_thr:
            width = width + 1
    return width

def significance(cc):
    '''
    Calculate the significance of each cross correlation sequence
    '''
    cc_mean = np.mean(cc)
    cc_std = np.std(cc)
    return (np.amax(cc) - cc_mean)/cc_std

In [24]:
total_time = 286
time = np.linspace(1,286,286)
time.T.shape
time = [time,time]
np.shape(np.array(time).T)

(286, 2)

In [ ]:
# If time range is given, say between 2020-01-08, 09:10 - 09:30
timeRange = ['2020-01-08_09_10',
             '2020-01-08_09_15',
             '2020-01-08_09_20',
             '2020-01-08_09_25',
             '2020-01-08_09_30'
            ]

# Extract any files that exsist within this time range. For testing, I am only using array 2 and array 3
for time in timeRange:
    path_in_str_list = []
    pathlist = Path('/Users/hyh/Desktop/Test Files/Postfiltered/').glob('*'+time+'*[2-3].raw')
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        path_in_str_list.append(path_in_str)
        path_in_str_list.sort()
    # Load data
    data1,samplerate = sf.read(path_in_str_list[0], 
                   channels=4, 
                   samplerate=16000,
                   subtype='PCM_16'
                  )
    data2,samplerate = sf.read(path_in_str_list[1], 
                   channels=4, 
                   samplerate=16000,
                   subtype='PCM_16'
                  )
    # Then do cross correlation

In [52]:
# Return the table 
Fs = 16000
total_time = int(np.minimum(len(data1), len(data2))/Fs)
window_len = 1 # how many seconds in a window
l = Fs * window_len
num_channel = 4
# Initialize empty arrays
time_vector = np.linspace(1,total_time, total_time)
delay_vector = np.zeros(total_time)
width_vector = np.zeros(total_time)
significance_vector = np.zeros(total_time)
table_pair = np.zeros([total_time, 4])
table = np.zeros([num_channel, num_channel, total_time, 4])

for channel1 in range(num_channel):
    for channel2 in range(num_channel):
        for ii in range(total_time):
            sig1, sig2 = truncate(data2[:,channel1], data2[:,channel2])
            sig1Temp, sig2Temp = sig1[ii * l:(ii + 1)* l], sig2[ii * l:(ii + 1)* l]
            tau, cc, R, SIG1, SIG2 = gcc_phat(sig1Temp, sig2Temp, fs=16000)
            width1 = width_cc(cc,0.5)
            significance1 = significance(cc)
            delay_vector[ii] = tau
            width_vector[ii] = width1
            significance_vector[ii] = significance1
        table_pair = np.array([time_vector, delay_vector, width_vector, significance_vector]).T
        table[channel1, channel2, :, :] = table_pair

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


In [58]:
np.save('table.npy',table)

In [3]:
data = np.load('table.npy')

In [4]:
data.shape

(4, 4, 286, 4)

In [77]:
# Convert data into 16 dataframes, represents 16 combinations between two microphones.
filename = "postfiltered_2020-01-08_09_25_03"
arrayA   = "1"
arrayB   = "2"
channels = [0,1,2,3]
columnNames = ["Delay","Width","Sig"]
combs = itertools.product(channels, repeat=2)

for aComb in combs:
    cha1, cha2= aComb
    tmp = data[cha1,cha2,:,:]
    # Generate a column that tells the combination
    combCol = [str(cha1)+str(cha2)] * tmp.shape[0]
    d = pd.DataFrame(tmp[:,1:], index=tmp[:,0], columns=columnNames)
    d.insert(0, "Comb", combCol, True) 
    d.to_csv("ccResults/"+filename+"_"+arrayA+arrayB+"_"+str(cha1)+str(cha2)+".csv", index=True)